In [1]:
import subprocess
from pydub import AudioSegment



def extract_audio_from_video(video_path, audio_path):

    command = [

        "ffmpeg",

        "-i",

        video_path,

        "-vn",

        audio_path,

    ]

    subprocess.run(command)


track = AudioSegment.from_mp3("./files/DP_Longest_Common_Subsequence_Leetcode1143.mp3")

track.duration_seconds